<a href="https://colab.research.google.com/github/sapienhwaker/Locality-Sensitive-Hashing/blob/main/LSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Prasad Hajare A20232707

## Prasad Hajare

Locality Sensitive Hashing

In [ ]:
!pip install pyspark

In [ ]:
#Importing relevent packages
import random
import os
import numpy as np
from random import randrange
from shutil import copyfile

import sys
import re
import collections
#from pyspark import SparkContext, SparkConf

## Execute the follwing two cells to generate your data sample

In [ ]:
MY_ID = '819'
#My_ID = '707'
# MY_ID should be string
# Example MY_ID='819'

task_dict = {0:'taska',1:'taskb',2:'taskc',3:'taskd',4:'taske'}


try:
    n_1 = int(MY_ID[-1]) % 5
    n_2 = int(MY_ID[-2]) % 5
    n_3 = int(MY_ID[-3]) % 5
    
    while n_1 == n_2 or n_1 == n_3:
        n_1 = (n_1 + 1) % 5
    
    while n_1 == n_2 or n_2 == n_3:
        n_2 = (n_2 + 1) % 5
        
except Exception as e:
    print('Please enter a valid ID...')

id_dict = {0:n_1,1:n_2,2:n_3}
print(id_dict)

{0: 4, 1: 1, 2: 3}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/My Drive/BIGDATA Fall2020/Project_2/corpus-20090418' 
# Edit this path if the data directory is not in the current directory

try:
    os.makedirs('Data_Sample')
    os.makedirs('Original_Sample')
except Exception as e:
    pass

for i in range(3):
    task = task_dict[id_dict[i]]
    
    for file in os.listdir(data_path):
        if task in file:
            if 'orig' in file:
                copyfile(data_path + '/' + file, 'Original_Sample/' + file)
            else:
                copyfile(data_path + '/' + file, 'Data_Sample/' + file)

Your dataset for this project will be in <b>Data_Sample</b>


Your query choices will be in <b>Original_Sample</b> directory


You have to use any one original Wikipedia article from <b>Original_Sample</b> for <b>Fact Check</b> steps

### STEP - 1: Shingling

In [ ]:
## function to get shingles from the sample documents
def get_sample_shingles(k, fileIndex):
  fileNumber = fileIndex
  data_sample_path = '/content/Data_Sample'
  for file in os.listdir(data_sample_path):
    file_address = data_sample_path + "/" + file
    try:
      f = open(file_address, 'rb')
      file_data = f.read().lower()
      templi = re.split(rb'\W+', file_data)
      #print("Processed file:", fileNumber)
      index = 0
      temp_set = set()
      while index < len(templi)-k:
        count = 0
        s = b" "
        space = b" "
        while count < k:
          s = b"".join([s, templi[index+count], space])
          count += 1
  
        index += 1
        temp_set.add(s.strip().lower())
        shingles_set.add(s.strip().lower())
      dictionary[fileNumber] = temp_set
      name_index[fileNumber] = file
      fileNumber += 1
    except:
      pass

In [ ]:
## Function to get shingles from original documents
def get_original_shingles(k, fileIndex):
  fileNumber = fileIndex
  original_sample_path = '/content/Original_Sample'
  for file in os.listdir(original_sample_path):
    file_address = original_sample_path + "/" + file
    try:
      f = open(file_address, 'rb')
      file_data = f.read().lower()
      templi = re.split(rb'\W+', file_data)
      #print("Processed file:", fileNumber)
      
      index = 0
      temp_set = set()
      while index < len(templi)-k:
        count = 0
        s = b" "
        space = b" "
        while count < k:
          s = b"".join([s, templi[index+count], space])
          count += 1
  
        index += 1
        temp_set.add(s.strip())
        shingles_set.add(s.strip())
      dictionary[fileNumber] = temp_set
      name_index[fileNumber] = file
      fileNumber += 1
    except:
      pass

In [ ]:
## Getting shingles for k =3, 4, 5

for k in range(3,6):
  dictionary = dict()
  shingles_set = set()
  name_index = dict()
  get_sample_shingles(k, 0)
  get_original_shingles(k, len(dictionary))
  print("k = " + str(k) + " : " + str(len(shingles_set)))
  #print(dictionary[46])

k = 3 : 7827
k = 4 : 8721
k = 5 : 9132


Report results (number of unique k-shingles) for k={3,4,5} below:
1. k=3: 7827
2. k=4: 8721
3. k=5: 9132

In [ ]:
## Indexing of shingles for k = 5
shingles_list = list(shingles_set)  #the index number in the list will be the index for each specific shingle
#print(shingles_list)

### STEP - 2: Min-Hashing

In [ ]:
# ***************************************************NEW***********************************************************
# Generate Hash functions - 
    # We use (ax + b) mod N formula to permute shingle index
    # where a,b are random numbers, N total index size, and x is the index
# We need to do L permutations - In other words we need to have L permutations (lists) of new indexes
# Following function takes total index size N and L as arguments
    # And returns L new lists of size N
    
def get_hash_functions(N,L):
    hash_functions = []
    
    for itr in range(L):
        a=randrange(1,400)
        b=randrange(1,400)
        
        new_hash_function = []
        for i in range(N):
            new_hash_function.append((a * i + b) % N)
        
        hash_functions.append(new_hash_function)
    return hash_functions
        
# test
#hash_functions = get_hash_functions(len(shingles_set),1000)

#### For L = 50

In [ ]:
### Generating Signature Matrix
# For L = 50
L = 50

hash_functions = get_hash_functions(len(shingles_set), L)
signature_matrix = np.full((L, len(dictionary)), np.inf)

for k in range(0, L):
  scrambled = hash_functions[k]
  for i in range(0, len(shingles_set)):
    current_index = scrambled[i]
    shingle = shingles_list[i]
    for j in range(0, len(dictionary)):
      if shingle in dictionary[j]:
        if current_index < signature_matrix[k][j]:
          signature_matrix[k][j] = current_index

In [ ]:
#signature_matrix

In [ ]:
## Getting Jaccard Similarity with one of the original documents with all the sample documents
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,58]
for i in range(0, 57):
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/L)

In [ ]:
#similarity_count

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comparing candidates with orig_taske.txt
['0.6 similarity in g4pB_taske.txt', '0.5 similarity in g2pB_taske.txt', '0.32 similarity in g4pC_taske.txt', '0.22 similarity in g1pB_taske.txt']


### For L = 100

In [ ]:
L = 100

hash_functions = get_hash_functions(len(shingles_set), L)
signature_matrix = np.full((L, len(dictionary)), np.inf)

for k in range(0, L):
  scrambled = hash_functions[k]
  for i in range(0, len(shingles_set)):
    current_index = scrambled[i]
    shingle = shingles_list[i]
    for j in range(0, len(dictionary)):
      if shingle in dictionary[j]:
        if current_index < signature_matrix[k][j]:
          signature_matrix[k][j] = current_index

In [ ]:
#signature_matrix

In [ ]:
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,59]
for i in range(0, 57):
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/L)

In [ ]:
#similarity_count

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comparing candidates with orig_taske.txt
['0.46 similarity in g4pE_taskb.txt', '0.34 similarity in g0pA_taskb.txt']


### For L = 200

In [ ]:
L = 200

hash_functions = get_hash_functions(len(shingles_set), L)
signature_matrix = np.full((L, len(dictionary)), np.inf)

for k in range(0, L):
  scrambled = hash_functions[k]
  for i in range(0, len(shingles_set)):
    current_index = scrambled[i]
    shingle = shingles_list[i]
    for j in range(0, len(dictionary)):
      if shingle in dictionary[j]:
        if current_index < signature_matrix[k][j]:
          signature_matrix[k][j] = current_index

In [ ]:
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,57]
for i in range(0, 57):
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/L)

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comparing candidates with orig_taskd.txt
g3pA_taskd.txt Similarity 0.91
g4pC_taskd.txt Similarity 0.685
g4pB_taskd.txt Similarity 0.21
g2pB_taskd.txt Similarity 0.43
g0pC_taskd.txt Similarity 0.425
g2pA_taskd.txt Similarity 0.285


### L = 500

In [ ]:
L = 500

hash_functions = get_hash_functions(len(shingles_set), L)
signature_matrix = np.full((L, len(dictionary)), np.inf)

for k in range(0, L):
  scrambled = hash_functions[k]
  for i in range(0, len(shingles_set)):
    current_index = scrambled[i]
    shingle = shingles_list[i]
    for j in range(0, len(dictionary)):
      if shingle in dictionary[j]:
        if current_index < signature_matrix[k][j]:
          signature_matrix[k][j] = current_index

In [ ]:
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,59]
for i in range(0, 57):
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/L)

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comparing candidates with orig_taskb.txt
g4pE_taskb.txt Similarity 0.352
g0pA_taskb.txt Similarity 0.366


### L = 1000

In [ ]:
L = 1000

hash_functions = get_hash_functions(len(shingles_set), L)
signature_matrix = np.full((L, len(dictionary)), np.inf)

for k in range(0, L):
  scrambled = hash_functions[k]
  for i in range(0, len(shingles_set)):
    current_index = scrambled[i]
    shingle = shingles_list[i]
    for j in range(0, len(dictionary)):
      if shingle in dictionary[j]:
        if current_index < signature_matrix[k][j]:
          signature_matrix[k][j] = current_index

In [ ]:
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,57]
for i in range(0, 57):
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/L)

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comparing candidates with orig_taskd.txt
g3pA_taskd.txt Similarity 0.923
g4pC_taskd.txt Similarity 0.705
g4pB_taskd.txt Similarity 0.229
g2pB_taskd.txt Similarity 0.493
g0pC_taskd.txt Similarity 0.333
g2pA_taskd.txt Similarity 0.205


### STEP - 3: LSH

In [ ]:
# r and B values t values
r = 20
B = 199
t = 0.2

In [ ]:
from numpy import random

x=random.randint(1000, size=(r))

In [ ]:
buckets = collections.defaultdict(set)

index = 0
temp_dict = collections.defaultdict(list)

while index < 1000:
  for doc_index in range(0, 57):
     band = signature_matrix[index:index+r, doc_index]
     bucket_id = np.sum(np.prod([band,x], axis=0)) % B
     temp_dict[bucket_id].append(doc_index)

  index += r
  for key, value in temp_dict.items():
    if len(value) > 1:
      for i in value:
        buckets[key].add(i)

In [ ]:
index = 0
candidate_docs = set()
for doc_index in range(59, 60):
  while index < 1000:
    band = signature_matrix[index:index+r, doc_index]
    bucket_id = np.sum(np.prod([band,x], axis=0)) % B
    li = buckets[bucket_id]
    candidate_docs.update(li)
    index = index + r
  index = 0

In [ ]:
print("Total number of candidate documents: " + str(len(candidate_docs)))

Total number of candidate documents: 57


In [ ]:
#candidate_docs

Report all documents (file_names) below that have Jaccard similarity > t=0.20
Sort the documents in decreasing order of the Jaccard similarity


In [ ]:
candidates_list = list(candidate_docs)
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,57]
for i in candidates_list:
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/1000)

In [ ]:
#similarity_count

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comaparing candidates with orig_taskd.txt


['0.923 similarity in document g3pA_taskd.txt',
 '0.705 similarity in document g4pC_taskd.txt',
 '0.229 similarity in document g4pB_taskd.txt',
 '0.493 similarity in document g2pB_taskd.txt',
 '0.333 similarity in document g0pC_taskd.txt']

Report the list of false positives and false negatives below



For L = 1000, b = 50, r = 20
Candidate documents obtained from LSH are 57
Hence, false positive = 51 and false negative = 0

In [ ]:
# Experiment with different values of b,r,B, and t
    # to reduce the number of false positives and false negatives
    # Report all results in a table in a separate word document

# r and B values t values
r = 50
B = 299
t = 0.2

In [ ]:
from numpy import random

x=random.randint(1000, size=(r))

In [ ]:
buckets = collections.defaultdict(set)

index = 0
temp_dict = collections.defaultdict(list)

while index < 1000:
  for doc_index in range(0, 57):
     band = signature_matrix[index:index+r, doc_index]
     bucket_id = np.sum(np.prod([band,x], axis=0)) % B
     temp_dict[bucket_id].append(doc_index)

  index += r
  for key, value in temp_dict.items():
    if len(value) > 1:
      for i in value:
        buckets[key].add(i)

In [ ]:
index = 0
candidate_docs = set()
for doc_index in range(59, 60):
  while index < 1000:
    band = signature_matrix[index:index+r, doc_index]
    bucket_id = np.sum(np.prod([band,x], axis=0)) % B
    li = buckets[bucket_id]
    candidate_docs.update(li)
    index = index + r
  index = 0

In [ ]:
print("Total number of candidate documents: " + str(len(candidate_docs)))

Total number of candidate documents: 42


In [ ]:
candidates_list = list(candidate_docs)
similarity_count = collections.defaultdict(float)
original = signature_matrix[:,57]
for i in candidates_list:
  another = signature_matrix[:,i]
  comparison = original == another
  count = np.count_nonzero(comparison)
  similarity_count[i] = (count/1000)

In [ ]:
## Documetns having Jaccard Similarity greater than 20%
answer = []
print("Comparing candidates with " + name_index[58])
for key, value in similarity_count.items():
  if value > 0.2:
    string = str(similarity_count[key]) + " similarity in " + name_index[key]
    answer.append(string)
    #print(name_index[key] + " Similarity " + str(similarity_count[key]))
answer.sort(reverse=True)
print(answer)

Comaparing candidates with orig_taskd.txt
g3pA_taskd.txt Similarity 0.923
g4pC_taskd.txt Similarity 0.705
g4pB_taskd.txt Similarity 0.229
g2pB_taskd.txt Similarity 0.493
g0pC_taskd.txt Similarity 0.333
